# Boeuf results

In [9]:
# Scientific libraries
import numpy as np
import scipy


# Graphic libraries
%matplotlib widget

import matplotlib.pyplot as plt

plt.style.use("presentation")
plt.rcParams["figure.figsize"] = (4, 3)

# Creating alias for magic commands

# LPPview Classes
from LPPview import *
from LPPview.Classes.LPPic_temporal import History
from LPPview.Classes.LPPic_fields import field
from LPPview.Classes.LPPic_fields import field as Field
from LPPview.Classes.LPPic_newwalls import newwalls as Newwalls
from LPPview.Classes.LPPic_temporal import History

from plasmapy.physics import Debye_length
from plasmapy.physics import plasma_frequency
from astropy import units as u

from scipy.ndimage import gaussian_filter1d as smooth
from scipy.ndimage import gaussian_filter

from tqdm import tqdm_notebook as tqdm

In [11]:
path_ref = "/DATA/tavant/266_Boeuf_166Thomas/"
path_L2 = "/DATA/tavant/158_Beauf_fakeR/"
path_L4 = "/DATA/tavant/163_Beauf_fakeR2/"
paths = [path_ref, path_L4, path_L2]
names = ["no $L_R$", "$L_R$=4cm", "$L_R$=2cm"]
colors = ["k","b", "r"]
fields = [Field(path) for path in paths ]

for f in fields:
    f.definecoords()
histories = [History(path)for path in paths ]
walls = [Newwalls(path) for path in paths ]



found 498 files
found 403 files
found 772 files
I've found a temporale file !
found 1 files
loading dat file
I've found a temporale file !
found 1 files
loading dat file
I've found a temporale file !
found 1 files
loading dat file
found 497 files
found 402 files
found 771 files


# Temporal evolutions

In [3]:
hDicts = [h.return_history() for h in histories]
print(hDicts[0].keys())

dict_keys(['time', 'elec', 'ions', 'SUMe_x', 'SUMe_y', 'SUMe_z', 'coll', 'ioni', 'mobi', 'elec_SEE', 'elec_SEE_sup', 'elec_wal', 'elec_cou', 'elec_Oz', 'I_cd', 'I_an', 'I_an_e', 'I_an_i', 'I_beam', 'I_out_electron'])


# Wave energy density 

In [24]:
def return_dni_ni(field : Field, i=-1):
    """Return the wave energy:
    $ dni =  std(E) \sqrt{2} $
    """
    tab = field.return_fromkey(i, "Numi")
    
    std_E = tab.std(axis=0)
    
    return np.sqrt(2)*std_E, tab.mean(axis=0)

def return_dni_ni_averaged(field:Field, tmin_mus=9, tmax_mus=10):
    """Average dni and ni over time """
    
    tmin_ind, tmax_ind = [int ( t_mus*1e-6 / (field._dT * field._Na) ) for t_mus in [tmin_mus, tmax_mus] ]
    
    for t in range(tmin_ind, tmax_ind):
        if t == tmin_ind:
            dni, ni = return_dni_ni(field, t)
            dni_norm = dni/ni
        else:
            dni_tmp, ni_tmp = return_dni_ni(field, t)
            
            dni += dni_tmp
            ni += ni_tmp
            dni_norm += dni_tmp/ni_tmp
        
    dni /= tmax_ind - tmin_ind
    ni /= tmax_ind - tmin_ind
    dni_norm /= tmax_ind - tmin_ind
    
    return dni, ni, dni_norm
                          

In [17]:
fig, axarr = plt.subplots(1, 2, figsize=(8, 3))

vmax = 0
for f, n in zip(fields, names):
    
    dni, ni = return_dni_ni(f, 400)

    axarr[0].plot(f.tab_y, dni, label=n)
    axarr[1].plot(f.tab_y, dni/ni*100, label=n)


for ax in axarr:
    ax.legend()    
    ax.set_xlabel("Axial position $z$ [cm]")

axarr[0].set_ylabel("Ion fluctuation $\delta n_i$ [SI]")
axarr[1].set_ylabel("Nomalized $\delta n_i / n_i$ [%]")
fig.suptitle("Ion density fluctuation at $t=10\mu$s ")
#plt.savefig("Boeuf_Ex_snapshot.png", dpi=400)

FigureCanvasNbAgg()

Text(0.5, 0.98, 'Ion density fluctuation at $t=10\\mu$s ')

In [51]:
fig, axarr = plt.subplots(1, 3, figsize=(8, 3))

vmax = 0
for f, n in zip(fields, names):
    dni, ni, dni_norm = return_dni_ni_averaged(f, tmin_mus=7, tmax_mus=10)

    axarr[0].plot(f.tab_y, dni, label=n)
    axarr[1].plot(f.tab_y, dni/ni*100, label=n)
    axarr[2].plot(f.tab_y, dni_norm*100, label=n)


for ax in axarr:
    ax.legend()    
    ax.set_xlabel("Axial position $z$ [cm]")

axarr[0].set_ylabel("Ion fluctuation $\delta n_i$ [SI]")
axarr[1].set_ylabel("Nomalized $<\delta n_i> /< n_i>$ [%]")
axarr[2].set_ylabel("Nomalized $<\delta n_i / n_i>$ [%]")
fig.suptitle("Ion density fluctuation averaged between $t=7$ and $10\mu$s ")
#plt.savefig("Boeuf_Ex_snapshot.png", dpi=400)

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

Text(0.5, 0.98, 'Ion density fluctuation averaged between $t=7$ and $10\\mu$s ')

# Wave amplitude evolution

In [37]:
def return_dni_2D_zt(field, nt_max=None):
    """return the spatio_temporal evolution of the wave energy"""
    if nt_max is None:
        nt_max = field._nT
        
    dni_2D = np.zeros( shape=( nt_max, field._ymax+1))
    ni_2D = np.zeros( shape=( nt_max, field._ymax+1))
    
    for i in tqdm(range(nt_max)):
        dni, ni = return_dni_ni(field, i)
        dni_2D[i] = dni
        ni_2D[i] = ni
        
    return dni_2D, ni_2D

In [38]:
nt_max = 9999
for f in fields:
    nt_max = min(nt_max, f._nT)
    
[dni2D_0, ni2D_0], [dni2D_1, ni2D_1], [dni2D_2, ni2D_2] = [return_dni_2D_zt(f, nt_max) for f in fields]


In [57]:
def dni_2D_evolution(smooth_gaussian_2D=False, normed=False):

    fig, axarr = plt.subplots(1, 3, figsize=(8, 3))

    icut = 10

    nt_max = 900
    for f in fields:
        nt_max = min( nt_max, f._nT)

    datas_dn = [dni2D_0, dni2D_1, dni2D_2]
    datas_n = [ni2D_0, ni2D_1, ni2D_2]

    """Process the data : 2D Gaussian smooth"""
    if smooth_gaussian_2D :
        datas = [ gaussian_filter(d, sigma=2, order=0) for d in datas_dn]
    else:
        datas = [ d for d in datas_dn]
        
    """Process the data : Normalized the fluctuations"""
    if normed:
        datas = [ dn/n*100 for dn, n in zip(datas, datas_n)]
        
        vmax = max( [ (d[-10:, :]).max() for d in datas])
        #vmax = 19
        print(vmax)
        
        cb_label= "fluctuations $\delta n_i / n_i$ [%]"

    else :
        datas = [data/data.max() for data in datas]
        vmax = 1
        cb_label= "fluctuations $\delta n_i$ [m$^{-3}$]"

    vmin = 0

    for f, ax, n, data in zip(fields, axarr, names, datas):
        f.definecoords()

        im =ax.imshow(data , extent=(0,f._Ly*100, 0, nt_max*f._dT*f._Na*1e6), vmin=vmin, vmax=vmax, cmap="plasma")
        ax.set_title(n, fontsize=12)
        ax.set_xlabel("Axial position $z$ [cm]")


    axarr[0].set_ylabel("Time [$\mu$s]")
    cb = plt.colorbar(im, ax=axarr[2], fraction=0.05, aspect=30, ticks=[0,vmax])
    cb.ax.set_ylabel(cb_label)
    #cb.ax.set_yticks([0,1])
    cb.ax.set_yticklabels([0, f"{vmax:3.1f}"])
    fig.suptitle("Axial-temporal evolution of $W$", fontsize=14)


dni_2D_evolution()
# plt.savefig("Boeuf_Fake_R_dni.png", dpi=400)


# dni_2D_evolution(True)
# plt.savefig("Boeuf_Fake_R_dni_smoothed.png", dpi=400)


dni_2D_evolution(normed=True)
# plt.savefig("Boeuf_Fake_R_dni_normed.png", dpi=400)


# dni_2D_evolution(True, normed=True)
# plt.savefig("Boeuf_Fake_R_dni_normed_smoothed.png", dpi=400)


/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


FigureCanvasNbAgg()

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


FigureCanvasNbAgg()

20.58450342877192


/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


## Conclusion
The Ion density fluctuations are not "simply" conserved and convected...